In [22]:
import pandas as pd
import yfinance as yf
import datetime

# Download the gold dataset from Yahoo Finance (5 years of data)
symbol = "GC=F"  # Symbol for gold futures
end_date = datetime.datetime.today().strftime('%Y-%m-%d')  # Today's date
start_date = (datetime.datetime.today() - pd.DateOffset(years=5)).strftime('%Y-%m-%d')  # 5 years ago

gold_data = yf.download(symbol, start=start_date, end=end_date)

# Flatten the MultiIndex columns and explicitly set the correct names
gold_data.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

# Calculate the 200-day moving average for the 'Close' price
gold_data['200_MA'] = gold_data['Close'].rolling(window=200).mean()

# Function to calculate weight based on the current price and 200-day moving average
def calculate_weight(close, ma):
    # Return 0 if ma is NaN or if it's an invalid value
    if pd.isna(ma) or ma == 0:  # Handle NaN or zero in moving average
        return 0
    diff_percentage = abs(close - ma) / ma * 100
    # Weight based on how close the price is to the 200-day MA
    if diff_percentage <= 10:  # Close to 200-day MA
        return 1
    elif diff_percentage <= 25:  # Slightly away from MA
        return 0.75
    elif diff_percentage <= 50:  # Moderately away from MA
        return 0.5
    else:  # Far away from MA
        return 0

# Apply the weight calculation for today (latest data in the dataset)
today_data = gold_data.iloc[-1]  # Get the last row (today's data)
close_price_today = today_data['Close']
ma_today = today_data['200_MA']

# Calculate the weight for today
weight_today = calculate_weight(close_price_today, ma_today)

# Set a budget (for example, $1000)
budget = 1000

# Calculate how many units of gold to buy today
units_to_buy_today = (budget * weight_today) / close_price_today

# Calculate the investment in USD
investment_in_usd = units_to_buy_today * close_price_today

# Output the results
print(f"Today's gold price: {close_price_today}")
print(f"200-day moving average: {ma_today}")
print(f"Weight based on price comparison: {weight_today}")
print(f"Units to buy today based on $1000 budget: {units_to_buy_today}")
print(f"Investment in USD to buy {units_to_buy_today} Amount to be invested: {investment_in_usd}")


[*********************100%***********************]  1 of 1 completed

Today's gold price: 2664.89990234375
200-day moving average: 2441.4454943847654
Weight based on price comparison: 1
Units to buy today based on $1000 budget: 0.3752486159500817
Investment in USD to buy 0.3752486159500817 Amount to be invested: 1000.0
